# Full Dataset KMeans

In [1]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import pandas as pd
from dask import delayed, persist
import numpy as np
from dask.distributed import Client
from glob import glob
import re
import nltk
import dask
import csv
csv.field_size_limit(10000000)


pd.set_option('display.max_columns', None) 

%matplotlib inline

csv.field_size_limit()

10000000

In [2]:
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client

Client Scheduler: tcp://127.0.0.1:59307 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 2.00 GB


In [3]:
import dask.array as da

x = da.random.random((10000, 10000), chunks='16 MiB')
x

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1250, 1250), chunktype=numpy.ndarray>

In [4]:
y = (x + x.T) - x.mean(axis=0)
y.sum().compute()

50003525.5173713

In [5]:
del y
del x

In [6]:
best_list = ['JobID', 'CleanJobTitle', 'CanonCity', 'CanonState', 'JobDate', 'JobText', 'Source', 'CanonEmployer',
             'Latitude', 'Longitude', 'CanonIntermediary', 'CanonJobTitle', 'CanonCounty', 'DivisionCode', 'MSA', 'LMA',
             'InternshipFlag', 'ConsolidatedONET', 'CanonSkillClusters', 'CanonSkills', 'IsDuplicate', 'CanonMinimumDegree', 
             'CanonRequiredDegrees', 'CIPCode', 'MinExperience', 'ConsolidatedInferredNAICS', 'BGTOcc', 'MaxAnnualSalary',
             'MaxHourlySalary', 'MinAnnualSalary', 'MinHourlySalary', 'YearsOfExperience', 'CanonJobHours', 'CanonJobType',
             'CanonPostalCode', 'CanonYearsOfExperienceCanonLevel', 'CanonYearsOfExperienceLevel', 'ConsolidatedTitle', 
             'Language', 'BGTSubOcc', 'ConsolidatedDegreeLevels', 'MaxDegreeLevel', 'MinDegreeLevel']

In [7]:
dtypes={'JobID': np.str, 'CleanJobTitle': np.str, 'JobDomain': np.str, 
        'CanonCity': np.str, 'CanonCountry': np.str, 'CanonState': np.str, 
        'JobText': np.str, 'JobURL': np.str, 'PostingHTML': np.float64, 
        'Source': np.str, 'JobReferenceID': np.str, 'Email': np.str, 
        'CanonEmployer': np.str, 'Latitude': np.str, 'Longitude': np.str, 
        'CanonIntermediary': np.str, 'Telephone': np.str, 'CanonJobTitle': np.str, 
        'CanonCounty': np.str, 'DivisionCode': np.str, 'MSA': np.str, 'LMA': np.str,
        'InternshipFlag': np.str, 'ConsolidatedONET': np.str, 'CanonCertification': np.str, 
        'CanonSkillClusters': np.str, 'CanonSkills': np.str, 'IsDuplicate': np.str, 
        'IsDuplicateOf': np.float64, 'CanonMaximumDegree': np.str, 'CanonMinimumDegree': np.str, 
        'CanonOtherDegrees': np.str, 'CanonPreferredDegrees': np.str,
        'CanonRequiredDegrees': np.str, 'CIPCode': np.str, 'StandardMajor': np.str, 
        'MaxExperience': np.str, 'MinExperience': np.str, 'ConsolidatedInferredNAICS': np.str, 
        'BGTOcc': np.str, 'MaxAnnualSalary': np.str, 'MaxHourlySalary': np.str, 
        'MinAnnualSalary': np.str, 'MinHourlySalary': np.str, 'YearsOfExperience': np.str, 
        'CanonJobHours': np.str, 'CanonJobType': np.str, 'CanonPostalCode': np.str, 
        'CanonYearsOfExperienceCanonLevel': np.str, 'CanonYearsOfExperienceLevel': np.str, 
        'ConsolidatedTitle': np.str, 'Language': np.str, 'BGTSubOcc': np.str, 'JobDate': np.str,
        'ConsolidatedDegreeLevels': np.str, 'MaxDegreeLevel': np.str, 'MinDegreeLevel': np.str
                       }

In [8]:
df = dd.read_csv('~/Dropbox/Burning Glass/Data/random_data/rand/rand_samp_20.csv', 
                 engine='python', 
                 dtype=dtypes,
                 sample=2_500_000,
                #  encoding='latin-1',
                #  encoding='utf-8',
                #  assume_missing=True,
                 error_bad_lines=False,
                #  blocksize=None,
                 usecols=best_list,
#                  parse_dates=['JobDate']
                )
df

Skipping line 585: unexpected end of data


,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
npartitions=35,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [9]:
%%time

# df = df.repartition(npartitions=25)
df = df[df['JobText'].notnull()]

CPU times: user 1.53 ms, sys: 73 µs, total: 1.61 ms
Wall time: 1.61 ms


In [10]:
%%time

df.head()

CPU times: user 74.9 ms, sys: 8.66 ms, total: 83.6 ms
Wall time: 1.94 s


,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
0,38807273782,Business Development Manager - Construction,Irvine,CA,2020-07-03,Business Development Manager - Construction\n\...,Job intermediary,NaN,33.7425,-117.747,XO Staffing Agency,Construction Manager,Orange,11244.0,31080: Metropolitan Statistical Area|348: Comb...,DV064204|MT063110,0,11202200,Sales: Business Development;Specialized Skills...,{'Business Development': 'Sales: Business Deve...,FALSE,Bachelor's,Bachelor's,520201|521401|522001,NaN,2362.0,11-2022.00,88000.0,42.31,63000.0,30.29,NaN,NaN,NaN,92602,NaN,NaN,Construction Manager,en,Business Development Manager,16,NaN,16.0
1,38765089544,Stihl Repair Technician,Tucson,AZ,2020-04-19,STIHL Repair Technician\n\nEquipmentLocator.co...,Job Board,Equipmentlocator Com,32.2161,-110.971,NaN,Repair Technician,Pima,NaN,46060: Metropolitan Statistical Area,MT044606,0,49907100,Specialized Skills|Specialized Skills,"{'Repair': 'Specialized Skills', 'Retail Indus...",FALSE,NaN,NaN,NaN,NaN,5241.0,49-9071.91,NaN,NaN,NaN,NaN,NaN,fulltime,permanent,85701,NaN,NaN,Repair Technician,en,Building and General Maintenance Technician,NaN,NaN,NaN
2,38726367466,Pediatrics In - Academic Pediatric Geneticist,Fresno,CA,2020-03-10,Pediatrics in CA - Academic Pediatric Genetici...,Job intermediary,NaN,36.841,-119.8,Comphealth,NaN,Fresno,NaN,23420: Metropolitan Statistical Area,MT062342,0,19102903,Science and Research: Genetics;Specialized Ski...,{'Genetics': 'Science and Research: Genetics;S...,FALSE,NaN,NaN,NaN,NaN,62.0,19-1020.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93650,NaN,NaN,"Academic Pediatric Geneticist, Information And...",en,Biologist,NaN,NaN,NaN
3,38813841551,Ui Developer At The Judge Group,Irving,TX,2020-07-13,"UI Developer at The Judge Group\n\nIrving, TX\...",Job intermediary,NaN,32.842,-96.9719,"The Judge Group , Incorporated",User Interface (UI) Developer,Dallas,19124.0,19100: Metropolitan Statistical Area|206: Comb...,DV481912|MT481910,0,15113400,Information Technology: JavaScript and jQuery;...,{'AngularJS': 'Information Technology: JavaScr...,FALSE,NaN,NaN,NaN,8.0,54.0,15-1134.93,NaN,NaN,NaN,NaN,Min 2 Years|min 1 Year|8-10 years,fulltime,temporary,75014,6+,high,User Interface (UI) Developer,en,User Interface (UI) Developer,NaN,NaN,NaN
4,38799678091,Cdl A Regional Truck Driver,Chestertown,MD,2020-06-20,CDL A Regional Truck Driver\n\nCowan Systems\n...,Recruiter,Cowan Systems,39.2169,-76.0767,NaN,Class A CDL Truck Driver,Kent,NaN,NaN,CN240290,0,53303200,Administration: Scheduling;Specialized Skills,{'Scheduling': 'Administration: Scheduling;Spe...,FALSE,NaN,NaN,NaN,3.0,484.0,53-3032.00,65000.0,31.25,54000.0,25.96,one year|three years|One year|3 years,NaN,NaN,21620,1-6,mid,Class A CDL Truck Driver,en,Regional Truck Driver,NaN,NaN,NaN


In [11]:
df.npartitions

35

In [12]:
%%time

with ProgressBar():
    missing_values = df.isna().sum()
    missing_count = ((missing_values / df.index.size) * 100)
    missing_count_pct = missing_count.compute()
    
# missing_count_pct

CPU times: user 2.33 s, sys: 224 ms, total: 2.55 s
Wall time: 42.7 s


In [13]:
missing_count_pct

JobID                                0.000000
CleanJobTitle                        0.008750
CanonCity                            0.533776
CanonState                           0.044819
JobDate                              0.000213
JobText                              0.000000
Source                               1.197740
CanonEmployer                       20.021385
Latitude                             0.493652
Longitude                            0.493011
CanonIntermediary                   87.850364
CanonJobTitle                       36.211367
CanonCounty                          0.539965
DivisionCode                        68.530933
MSA                                  2.950172
LMA                                  0.844736
InternshipFlag                       0.004482
ConsolidatedONET                     3.585537
CanonSkillClusters                   5.107470
CanonSkills                          0.004482
IsDuplicate                          0.004909
CanonMinimumDegree                

In [18]:
import pyarrow

In [14]:
with ProgressBar():
    unique_occus = df.BGTOcc.unique().compute()
    print(unique_occus[:5])

0    11-2022.00
1    49-9071.91
2    19-1020.01
3    15-1134.93
4    53-3032.00
Name: BGTOcc, dtype: object


In [15]:
client.close()

In [15]:
df.tail()

,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
499995,38687559870,Ct Technologist | Radiology/Ct | | With Rotati...,Eau Claire,WI,2019-12-29,CT Technologist | Radiology/CT | 40hrs/wk | Va...,Company,St Nicholas Hospital,44.7985,-91.4824,NaN,Radiologic Technologist,Eau Claire,NaN,20740: Metropolitan Statistical Area,MT552074,0,29203400,Health Care: Emergency and Intensive Care;Spec...,{'Life Support': 'Health Care: Emergency and I...,FALSE,Bachelor's,NaN,NaN,NaN,6221.0,29-2099.92,NaN,NaN,NaN,NaN,1 Year,fulltime,permanent,54701,NaN,NaN,Radiologic Technologist,en,Radiologic Technician / Technologist,16,16.0,16.0
499996,38669407953,Dishwasher,Morrow,GA,2019-11-24,"Dishwasher\n\nChilis\n\n-\n\nMorrow, GA 30260\...",Job Board,Chili's,33.5854,-84.3271,NaN,Dish Washer,Clayton,NaN,12060: Metropolitan Statistical Area,MT131206,0,35902100,Personal Care and Services: Food and Beverage ...,{'Cooking': 'Personal Care and Services: Food ...,FALSE,NaN,NaN,NaN,NaN,722511.0,35-9021.00,NaN,NaN,NaN,NaN,NaN,fulltime,permanent,30260,0,zero,Dish Washer,en,Dishwasher,NaN,NaN,NaN
499997,38635347958,Music Therapist,Des Moines,IA,2019-10-20,Job Information\n\nYouth Emergency Services & ...,Job Board,Youth Emergency Services & Shelter,41.605,-93.6319,NaN,NaN,Polk,NaN,19780: Metropolitan Statistical Area,MT191978,0,29112500,Common Skills|Marketing and Public Relations: ...,"{'Communication Skills': 'Common Skills', 'Com...",FALSE,Bachelor's,Bachelor's,512305,2.0,62.0,29-1125.00,NaN,NaN,NaN,NaN,Minimum of two years,fulltime,permanent,50301,1-6,mid,Music Therapist,en,Art / Music / Recreational Therapist,16,NaN,16.0
499998,38595586361,Cdl A Flatbed Driver - Regional And Over The Road,New Orleans,LA,2019-10-02,CDL A Flatbed Drivers - Regional and OTR\n\nDe...,Job Board,Decker Truck Line,29.9575,-90.0769,NaN,Class A CDL Truck Driver,Orleans,NaN,35380: Metropolitan Statistical Area,MT223538,0,53303200,NaN,{},FALSE,NaN,NaN,NaN,NaN,484.0,53-3032.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70112,NaN,NaN,Class A CDL Truck Driver,en,Over-the-Road (OTR) Truck Driver,NaN,NaN,NaN
499999,38669475936,Azure Engineer,Schaumburg,IL,2019-11-24,"Azure Engineer\n\nYork Solutions, LLC Schaumbu...",Job intermediary,NaN,42.033,-88.0834,York Solutions,Azure Engineer,Cook,16974.0,16980: Metropolitan Statistical Area|176: Comb...,DV171697|MT171698,0,17219900,Information Technology: Distributed Computing;...,{'Apache Kafka': 'Information Technology: Dist...,FALSE,NaN,NaN,NaN,NaN,NaN,NaN,133819.0,64.34,133819.0,64.34,NaN,fulltime,permanent,60159,NaN,NaN,Azure Engineer,en,NaN,NaN,NaN,NaN


In [17]:
with ProgressBar():
    print(df.map_partitions(len).compute())

[########################################] | 100% Completed |  0.1s
0     20000
1     20000
2     20000
3     20000
4     20000
5     20000
6     20000
7     19999
8     20000
9     20000
10    20000
11    20000
12    20000
13    20000
14    20000
15    20000
16    20000
17    20000
18    20000
19    20000
20    20000
21    20000
22    20000
23    20000
24    20000
dtype: int64
